# Cleaning Kangaroo dataset

In [1522]:
import pandas as pd

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

from peb_util import map_label_to_kwh
from outlier_util import remove_outliers_iqr

In [1523]:
df = pd.read_csv('./data/Kangaroo.csv')

### Drop duplicate rows

In [1524]:
df.drop_duplicates('id', inplace=True)

Drop useless columns

In [1525]:
df.drop(columns=['Unnamed: 0', 'id', 'url', 'type', 'locality', 'monthlyCost', 'hasBalcony', 'accessibleDisabledPeople', 'kitchenSurface', 'hasTerrace', 'hasGarden', 'gardenOrientation', 'roomCount', 'streetFacadeWidth', 'floorCount', 'floodZoneType', 'terraceOrientation', 'hasAttic', 'hasBasement', 'diningRoomSurface', 'hasDiningRoom', 'hasLift', 'heatingType', 'hasLivingRoom', 'livingRoomSurface', 'gardenSurface', 'parkingCountIndoor', 'hasAirConditioning', 'hasArmoredDoor', 'hasVisiophone', 'bathroomCount'], inplace=True)

Replace values

In [1526]:

df['hasOffice'].replace(pd.NA, False, inplace=True)
df['hasPhotovoltaicPanels'].replace(pd.NA, False, inplace=True)
df['hasHeatPump'].replace(pd.NA, False, inplace=True)
df['hasThermicPanels'].replace(pd.NA, False, inplace=True)
df['hasFireplace'].replace(pd.NA, False, inplace=True)
df['hasDressingRoom'].replace(pd.NA, False, inplace=True)
df['hasSwimmingPool'].replace(pd.NA, False, inplace=True)

df['terraceSurface'].fillna(0.0, inplace=True)
df['parkingCountOutdoor'].fillna(0, inplace=True)
df['landSurface'].fillna(0.0, inplace=True)

Replace True_False with 1_0

In [1527]:
# Convert boolean columns to integers (True -> 1, False -> 0)
bool_columns = df.select_dtypes(include='bool').columns
for column in bool_columns:
    df[column] = df[column].astype(int)

# Verify the conversion
print(f"Converted {len(bool_columns)} boolean columns to integers:")
print(bool_columns.tolist())
print("\nDataFrame dtypes after conversion:")
print(df.dtypes.value_counts())

Converted 7 boolean columns to integers:
['hasDressingRoom', 'hasHeatPump', 'hasPhotovoltaicPanels', 'hasThermicPanels', 'hasOffice', 'hasSwimmingPool', 'hasFireplace']

DataFrame dtypes after conversion:
float64    9
int64      8
object     5
Name: count, dtype: int64


Strip & Uppercase strings

In [1528]:
string_columns = df.select_dtypes(include='object').columns

for column in string_columns:
    # Apply strip and uppercase, but only to non-NaN string values
    df[column] = df[column].apply(lambda x: x.strip().upper() if isinstance(x, str) else x)
    
    # Count the modified values
    modified_count = df[column].notna().sum()
    
    # Print the result
    print(f"Processed column '{column}': {modified_count} values modified")

Processed column 'subtype': 80367 values modified
Processed column 'province': 80367 values modified
Processed column 'buildingCondition': 57991 values modified
Processed column 'kitchenType': 41916 values modified
Processed column 'epcScore': 65390 values modified


Convert PEB before removing province col

In [1529]:
df["epc_kwh"] = df.apply(map_label_to_kwh, axis=1)
df.drop(columns=['epcScore', 'province'], inplace=True)

Remove Outliers first

In [1530]:
columns_for_outlier_removal = [
    'price',
    'habitableSurface'
]

min_thresholds = { 'habitableSurface' : 25 }  

df = remove_outliers_iqr(df, columns_for_outlier_removal, multiplier=1.5, min_thresholds=min_thresholds)


Total rows before removing outliers: 80367
Total rows after removing outliers: 59223
Total outliers removed: 21144
Percentage of data retained: 73.7%

Outliers removed per column:
  price: 6047 (7.5%)
  habitableSurface: 2998 (3.7%)


### Imputing values after removing outliers

In [1531]:
df['facedeCount'].fillna(int(df['facedeCount'].median()), inplace=True)
df['buildingConstructionYear'].fillna(int(df['buildingConstructionYear'].median()), inplace=True)
df['habitableSurface'].fillna(int(df['habitableSurface'].median()), inplace=True)
df['bedroomCount'].fillna(int(df['bedroomCount'].median()), inplace=True)
df['epc_kwh'].fillna(int(df['epc_kwh'].median()), inplace=True)

Clamp facade count to the range 1-4

In [1532]:
# # Clamp facedeCount to the range 1-4
# print(f"Rows with facedeCount > 4 before clamping: {sum(df['facedeCount'] > 4)}")
# df['facedeCount'] = df['facedeCount'].clip(lower=1, upper=4)
# print(f"Max facedeCount after clamping: {df['facedeCount'].max()}")
# print(f"Min facedeCount after clamping: {df['facedeCount'].min()}")

### Converting values

Find trend and convert Building Condition according to it

In [1533]:
building_condition_price = df.groupby('buildingCondition')['price'].mean().round().sort_values(ascending=False)
df['buildingCondition'] = df['buildingCondition'].map(building_condition_price.to_dict())

Find trend for postalCode

In [ ]:
# Calculate the average price/m2 per postalCode
df['price_per_m2'] = df['price'] / df['habitableSurface']

postal_code_price = df.groupby('postCode')['price_per_m2'].mean().round().sort_values(ascending=False)

df['postCode'] = df['postCode'].map(postal_code_price.to_dict())
df.drop(columns=['price_per_m2'], inplace=True)


KeyError: 'Columns not found: 140.625, 154.62886597938143, 161.29032258064515, 198.64666666666668, 239.996, 250.0, 261.1940298507463, 263.3333333333333, 264.70588235294116, 288.8888888888889, 291.5129151291513, 293.52941176470586, 299.4011976047904, 303.9647577092511, 304.4871794871795, 305.8103975535168, 304.05405405405406, 308.8888888888889, 312.5, 320.0, 325.2032520325203, 327.10280373831773, 333.3333333333333, 338.34586466165416, 338.9830508474576, 347.5882352941176, 347.82608695652175, 350.0, 357.14285714285717, 364.8068669527897, 366.6666666666667, 367.6470588235294, 373.8317757009346, 375.49407114624506, 379.746835443038, 380.7692307692308, 381.67175572519085, 383.06451612903226, 390.35087719298247, 390.625, 392.15686274509807, 393.3333333333333, 394.42231075697214, 395.56962025316454, 396.0, 395.8333333333333, 404.0816326530612, 404.76190476190476, 404.8582995951417, 408.3333333333333, 409.72222222222223, 409.8360655737705, 412.37113402061857, 413.1736526946108, 415.78947368421

Find trend and convert SubType according to it

In [ ]:
subtype_price = df.groupby('subtype')['price'].mean().round().sort_values(ascending=False)
df['subtype'] = df['subtype'].map(subtype_price.to_dict())

Find trend and convert KitchenType according to it

In [ ]:
kitchen_type_price = df.groupby('kitchenType')['price'].mean().round().sort_values(ascending=False)
df['kitchenType'] = df['kitchenType'].map(kitchen_type_price.to_dict())

Replace by mode

In [ ]:
df['buildingCondition'].fillna(df['buildingCondition'].mode()[0], inplace=True)
df['subtype'].fillna(df['subtype'].mode()[0], inplace=True)
df['kitchenType'].fillna(df['kitchenType'].mode()[0], inplace=True)

Drop type & dropna

In [ ]:
df.dropna(inplace=True)

price_corr = df.corr()['price'].sort_values(ascending=False)
print("Correlation with price:")
print(price_corr)

Correlation with price:
price                       1.000000
habitableSurface            0.522432
toiletCount                 0.441864
bedroomCount                0.382868
postCode                    0.371228
subtype                     0.338972
kitchenType                 0.282991
buildingCondition           0.251002
hasPhotovoltaicPanels       0.208022
hasHeatPump                 0.186513
facedeCount                 0.183534
landSurface                 0.180497
hasOffice                   0.171144
terraceSurface              0.128613
hasDressingRoom             0.128157
parkingCountOutdoor         0.126360
hasFireplace                0.125721
buildingConstructionYear    0.125469
hasThermicPanels            0.118150
hasSwimmingPool             0.114555
epc_kwh                    -0.226562
Name: price, dtype: float64


### Finally save the cleaned dataset

In [ ]:
# Save the cleaned dataset to CSV 
df.to_csv('./data/kangaroo-cleaned.csv', index=False)
print(f"Dataset saved with {len(df)} rows and {len(df.columns)} columns")

Dataset saved with 44809 rows and 21 columns
